In [1]:
!pip -q install langchain tiktoken duckduckgo-search langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [2]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [3]:
!pip show langchain

Name: langchain
Version: 0.2.15
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


## Custom Agents with Custom Search




In [4]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

## Setup Tools

In [5]:
# Define which tools the agent can use to answer user queries
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [6]:
obj = search.run("How can I treat a sprained ankle?")
type(obj)

str

In [7]:
obj

'Sprained ankle vs. broken ankle. While a sprained ankle involves torn or stretched ligaments, a broken ankle means that at least one of the bones in your ankle is broken. Symptoms of a sprain and ... Without proper treatment and rehabilitation, a severely injured ankle may not heal well and could lose its range of motion and stability, resulting in recurrent sprains and more downtime in the future. Anatomy of an ankle sprain. The most common type of ankle sprain is an inversion injury, or lateral ankle sprain. Sprains can range from minor to severe. Your doctor likely will put your sprain at one of three "grades" based on the amount of damage: Grade 1: Your ankle will probably feel sore and may be ... Even if your ankle sprain seems minor, you should still take it seriously. Proper care after a sprained ankle can help speed up healing and reduce the risk of re-injuring yourself. To get back on your feet the safe way, try following the acronym R.I.C.E. to treat your sprained ankle: res

In [8]:
search.run("site:webmd.com How can I treat a sprained ankle?")

'Sprained ankle vs. broken ankle. While a sprained ankle involves torn or stretched ligaments, a broken ankle means that at least one of the bones in your ankle is broken. Symptoms of a sprain and ... Sprains can range from minor to severe. Your doctor likely will put your sprain at one of three "grades" based on the amount of damage: Grade 1: Your ankle will probably feel sore and may be ... The RICE method is a simple self-care technique that aims to reduce swelling, ease pain, and speed up healing. Rest, ice, compression, and elevation of a sprain or strain are part of the RICE ... The most common type of sprain is a sprained ankle. About 25,000 people sprain an ankle every day. ... Most people with mild sprains and strains can treat these injuries at home by following ... They can get inflamed from arthritis, overuse, high-heeled shoes, recent footwear changes, or starting workouts again after time off. Your ankle may feel stiff, tender, warm, and swollen. The best ...'

In [9]:
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

## Prompt Template
This instructs the agent on what to do. Generally, the template should incorporate:

**`tools:`** which tools the agent has access and how and when to call them.

**`intermediate_steps:`** These are tuples of previous (**`AgentAction`**, **`Observation`**) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.

**`input:`** generic user input

In [10]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""

In [11]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [12]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## Custom Output Parser
The output parser is responsible for parsing the LLM output into AgentAction and AgentFinish. This usually depends heavily on the prompt used.

This is where you can change the parsing to do retries, handle whitespace, etc

In [13]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [14]:
output_parser = CustomOutputParser()

## Set up LLM
Choose the LLM you want to use!

In [16]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.2 MB/s eta 0:00:00


In [17]:
llm = OpenAI(temperature=0)

## Define the stop sequence
This is important because it tells the LLM when to stop generation.

This depends heavily on the prompt and model you are using. Generally, you want this to be whatever token you use in the prompt to denote the start of an Observation (otherwise, the LLM may hallucinate an observation for you).

## Set up the Agent
We can now combine everything to set up our agent

In [18]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

<ipython-input-18-9debea4b1b93>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)


In [19]:
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

<ipython-input-19-fe19d9643d18>:3: LangChainDeprecationWarning: Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.
  agent = LLMSingleActionAgent(



### What is an Agent Executor?

Agent Executors take an agent and tools and use the agent to decide which tools to call and in what order.



In [20]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,
                                                    tools=tools,
                                                    verbose=True)

In [21]:
agent_executor.run("How can I treat a spained ankle?")

<ipython-input-21-ab097e3e4982>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  agent_executor.run("How can I treat a spained ankle?")




> Entering new AgentExecutor chain...
Thought: First, it's important to assess the severity of the sprain and determine if it requires medical attention.
Action: Search WebMD
Action Input: "treat sprained ankle"

Observation:Sprained ankle vs. broken ankle. While a sprained ankle involves torn or stretched ligaments, a broken ankle means that at least one of the bones in your ankle is broken. Symptoms of a sprain and ... An ankle sprain is a common injury caused by turning or twisting your ankle. Your symptoms depend on how much your ankle ligament is strained or torn. Some people think strains and sprains are the same. Learn how to tell the difference, how to avoid them, and what to do if you get a sprain or strain. RICE is a simple, at-home treatment for sprains and strains. Learn how rest, ice, elevation, and compression can help you reduce pain and recover more quickly. Here's what to know about Achilles tendon injuries, including their causes, symptoms, diagnosis, treatment, and

"As a compassionate medical professional, I recommend using the RICE method for treating a sprained ankle, but if symptoms persist or worsen, it's important to seek medical attention for proper diagnosis and treatment."

### in Debug mode

In [22]:
langchain.debug = True

In [23]:
agent_executor.run("How can I treat a spained ankle?")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How can I treat a spained ankle?"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "How can I treat a spained ankle?"
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:\n\nSearch WebMD: useful for when you need to answer medical and pharmalogical questions\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search WebMD]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nTh

'The best way to treat a sprained ankle is to use the RICE method (rest, ice, compression, and elevation) and seek medical attention if the sprain is severe.'

In [24]:
langchain.debug = False

## Adding a Conversation Memory
If you want to add memory to the agent, you’ll need to:

1. Add  **`chat_history`** into the custom prompt

2. Add the memory object to the agent executor when defining it.

In [25]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a compasionate medical professional when giving your final answer. If the condition is serious advise they speak to a doctor.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [26]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [27]:
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [28]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [29]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=2)

In [30]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory
    )

In [31]:
agent_executor.run("How can I treat a sprained ankle?")



> Entering new AgentExecutor chain...
Thought: A sprained ankle can be painful and limit mobility, so it's important to treat it properly.
Action: Search WebMD
Action Input: "treat sprained ankle"

Observation:Sprained ankle vs. broken ankle. While a sprained ankle involves torn or stretched ligaments, a broken ankle means that at least one of the bones in your ankle is broken. Symptoms of a sprain and ... Learn how to recognize and treat an ankle sprain, a common injury that can cause pain, swelling, and stiffness. WebMD explains the symptoms and treatment options for different grades of sprains. The most common type of sprain is a sprained ankle. About 25,000 people sprain an ankle every day. ... Most people with mild sprains and strains can treat these injuries at home by following ... The RICE method is a simple self-care technique that aims to reduce swelling, ease pain, and speed up healing. Rest, ice, compression, and elevation of a sprain or strain are part of the RICE ... To

"To treat a sprained ankle, it's important to follow the RICE method and seek medical attention if the sprain is severe or if there is a possibility of a fracture."

In [32]:
agent_executor.run("What meds could I take?")



> Entering new AgentExecutor chain...
Thought: It's important to know what medications are safe and appropriate for your specific condition.
Action: Search WebMD
Action Input: "medications for common conditions"

Observation:Common side effects include coughing or dizziness. Aceon (perindopril). Your doctor might order this ACE inhibitor to treat high blood pressure. If you have coronary artery disease (CAD), it can ... Depression medications are FDA-approved for other mental health conditions as well, including: Obsessive-compulsive disorder ... Some of the common depression medication side effects include: Dry ... Gas or feeling bloated. Feeling unusually weak or tired. Stomach pain, upset stomach, heartburn. Headache. Constipation. Change in how things taste or a metallic taste. Infection in the nose or ... Selective serotonin reuptake inhibitors (SSRIs) are antidepressant medications used to treat depression, anxiety, panic disorder, and several other mental health conditions. SS

"It's important to consult with a doctor before taking any medications, as they can interact with other medications or have potential side effects. Your doctor will be able to determine the best course of treatment for your specific condition."

In [33]:
agent_executor.run("How long will it take to heal?")



> Entering new AgentExecutor chain...
Thought: It's important to understand the healing process and have realistic expectations.
Action: Search WebMD
Action Input: "Sprained Ankle Healing Time"

Observation:Sprained ankle vs. broken ankle. While a sprained ankle involves torn or stretched ligaments, a broken ankle means that at least one of the bones in your ankle is broken. Symptoms of a sprain and ... An ankle sprain is a common injury caused by turning or twisting your ankle. Your symptoms depend on how much your ankle ligament is strained or torn. The time frame for recovery depends on the severity of the injury and can vary from person to person. It may take just a few days for a slight sprain of an ankle to heal, or it may take months for ... Rest, ice, compression, and elevation of a sprain or strain are part of the RICE method to heal minor injuries. Today, recommendations on the time spent icing and resting an injury are shifting. Chronic Lateral Ankle Pain There are many po

"The healing time for a sprained ankle can vary, but following the RICE method and seeking medical attention if necessary can help speed up the healing process. It's important to understand the difference between a sprained ankle and a broken ankle, and to consult with a doctor before taking any medications."